In [1]:
import sys
from pathlib import Path   
root_dir = str(Path().absolute().parent)

if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

Adding the following directory to the PYTHONPATH: d:\Projects\ML\personalized-recommendation-sysyem\personalized-recommendation-system


In [2]:
%load_ext autoreload
%autoreload 2

import warnings

warnings.filterwarnings("ignore")

from loguru import logger

from recsys import hopsworks_integration, training
from recsys.config import settings

In [3]:
from pprint import pprint

pprint(dict(settings))

{'CUSTOMER_DATA_SIZE': <CustomerDatasetSize.SMALL: 'SMALL'>,
 'CUSTOM_HOPSWORKS_INFERENCE_ENV': 'custom_env_name',
 'FEATURES_EMBEDDING_MODEL_ID': 'all-MiniLM-L6-v2',
 'HOPSWORKS_API_KEY': SecretStr('**********'),
 'OPENAI_API_KEY': SecretStr(''),
 'OPENAI_MODEL_ID': 'gpt-4o-mini',
 'RANKING_DATASET_VALIDATON_SPLIT_SIZE': 0.1,
 'RANKING_EARLY_STOPPING_ROUNDS': 5,
 'RANKING_ITERATIONS': 100,
 'RANKING_LEARNING_RATE': 0.2,
 'RANKING_MODEL_TYPE': 'ranking',
 'RANKING_SCALE_POS_WEIGHT': 10,
 'RECSYS_DIR': WindowsPath('d:/Projects/ML/personalized-recommendation-sysyem/personalized-recommendation-system/recsys'),
 'TWO_TOWER_DATASET_TEST_SPLIT_SIZE': 0.1,
 'TWO_TOWER_DATASET_VALIDATON_SPLIT_SIZE': 0.1,
 'TWO_TOWER_LEARNING_RATE': 0.001,
 'TWO_TOWER_MODEL_BATCH_SIZE': 2048,
 'TWO_TOWER_MODEL_EMBEDDING_SIZE': 16,
 'TWO_TOWER_NUM_EPOCHS': 5,
 'TWO_TOWER_WEIGHT_DECAY': 0.001}


In [4]:
project, fs = hopsworks_integration.get_feature_store()

2025-03-15 16:31:37.869 | INFO     | recsys.hopsworks_integration.feature_store:get_feature_store:13 - Loging to Hopsworks using HOPSWORKS_API_KEY env var.


2025-03-15 16:31:37,871 INFO: Initializing external client
2025-03-15 16:31:37,872 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-15 16:31:41,830 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213659


In [5]:
feature_view_ranking = hopsworks_integration.feature_store.create_ranking_feature_views(fs)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1213659/fs/1201283/fv/ranking/version/1


In [6]:
X_train, X_val, y_train, y_val = feature_view_ranking.train_test_split(
    test_size=settings.RANKING_DATASET_VALIDATON_SPLIT_SIZE,
    description="Ranking training dataset",
)
X_train.head(3)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.24s) 


,age,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,index_name,index_group_name,section_name,garment_group_name,month_sin,month_cos
0,22.0,Sweater,Garment Upper body,Melange,Dark Red,Dark,Red,Tops Knitwear,Divided,Divided,Divided Collection,Knitwear,-1.000000,-1.836970e-16
1,33.0,T-shirt,Garment Upper body,All over pattern,Beige,Medium Dusty,Beige,Jersey,Ladieswear,Ladieswear,Womens Casual,Jersey Fancy,-1.000000,-1.836970e-16
3,39.0,Shorts,Garment Lower body,Denim,Dark Grey,Dark,Grey,Shorts,Ladieswear,Ladieswear,Womens Everyday Collection,Shorts,-0.866025,5.000000e-01


In [9]:
X_train.shape

(9036, 14)

In [7]:
y_train.head(3)

,label
0,1
1,1
3,1


In [8]:
model = training.ranking.RankingModelFactory.build()
trainer = training.ranking.RankingModelTrainer(
    model=model, train_dataset=(X_train, y_train), eval_dataset=(X_val, y_val)
)

In [10]:
trainer.fit()

0:	learn: 0.5141313	test: 0.5135087	best: 0.5135087 (0)	total: 182ms	remaining: 18s
1:	learn: 0.3931784	test: 0.3926257	best: 0.3926257 (1)	total: 197ms	remaining: 9.63s
2:	learn: 0.3066654	test: 0.3059594	best: 0.3059594 (2)	total: 220ms	remaining: 7.12s
3:	learn: 0.2423266	test: 0.2416390	best: 0.2416390 (3)	total: 248ms	remaining: 5.95s
4:	learn: 0.1931421	test: 0.1923695	best: 0.1923695 (4)	total: 271ms	remaining: 5.15s
5:	learn: 0.1548219	test: 0.1541259	best: 0.1541259 (5)	total: 284ms	remaining: 4.45s
6:	learn: 0.1249025	test: 0.1242005	best: 0.1242005 (6)	total: 314ms	remaining: 4.17s
7:	learn: 0.1015362	test: 0.1007581	best: 0.1007581 (7)	total: 369ms	remaining: 4.25s
8:	learn: 0.0831523	test: 0.0821063	best: 0.0821063 (8)	total: 417ms	remaining: 4.22s
9:	learn: 0.0677472	test: 0.0667419	best: 0.0667419 (9)	total: 441ms	remaining: 3.96s
10:	learn: 0.0552756	test: 0.0543664	best: 0.0543664 (10)	total: 455ms	remaining: 3.68s
11:	learn: 0.0455097	test: 0.0445764	best: 0.0445764 (

In [11]:
metrics = trainer.evaluate(log=True)

2025-03-15 16:39:15.726 | INFO     | recsys.training.ranking:evaluate:62 -               precision    recall  f1-score   support

           0       1.00      1.00      1.00       895
           1       1.00      1.00      1.00       110

    accuracy                           1.00      1005
   macro avg       1.00      1.00      1.00      1005
weighted avg       1.00      1.00      1.00      1005



In [12]:
trainer.get_feature_importance()

{'month_cos': 53.21534375629503,
 'month_sin': 43.969178636774934,
 'department_name': 0.6676404597136671,
 'index_name': 0.5054006212947384,
 'perceived_colour_master_name': 0.40350924702183066,
 'age': 0.34692670401411696,
 'garment_group_name': 0.3130302439398821,
 'perceived_colour_value_name': 0.24080757862480662,
 'index_group_name': 0.19359730829862362,
 'section_name': 0.10992847937244066,
 'product_group_name': 0.03259730951552113,
 'graphical_appearance_name': 0.002039655134412033,
 'product_type_name': 0.0,
 'colour_group_name': 0.0}

In [13]:
mr = project.get_model_registry()

In [15]:
ranking_module = hopsworks_integration.ranking_serving.HopsworksRankingModel(
    model=model
)
ranking_module.register(mr, feature_view_ranking, X_train, metrics)

Uploading: 100.000%|██████████| 495813/495813 elapsed<00:19 remaining<00:00  1.59s/it]
Uploading: 100.000%|██████████| 478/478 elapsed<00:03 remaining<00:00<00:49, 12.33s/it]
Model export complete: 100%|██████████| 6/6 [00:31<00:00,  5.33s/it]                   

Model created, explore it at https://c.app.hopsworks.ai:443/p/1213659/models/ranking_model/1
